In [51]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool,CatBoost, CatBoostRegressor
from collections import Counter
from keras.layers import Dense, Dropout, Embedding, Input, Reshape, Concatenate, Input, Flatten, LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import regularizers
import json
import re

In [2]:
def jsons_name_type(jsons_name):
    if not jsons_name:
        return -1
    if "Цифра дня" in jsons_name:
        return 1
    if "AliExpress" in jsons_name:
        return 2 
    if "Тинькофф" in jsons_name or "Tinkoff" in jsons_name:
        return 3
    if "ЧМ" in jsons_name:
        return 4
    if "фильм"  in jsons_name or "смотрет"  in jsons_name:
        return 5
    if "кредит"  in jsons_name:
        return 6
    if "инвестиц"  in jsons_name:
        return 7
    if "₽"  in jsons_name or "$"  in jsons_name:
        return 8
    if "тпуск"  in jsons_name:
        return 9
    return 0

In [3]:
data_path = 'dataset/'
customer_train = pd.read_csv(f'{data_path}customer_train.csv')
customer_test = pd.read_csv(f'{data_path}customer_test.csv')
stories_description = pd.read_csv(f'{data_path}stories_description.csv')
stories_reaction_test = pd.read_csv(f'{data_path}stories_reaction_test.csv')
stories_reaction_test['event_dttm']=pd.to_datetime(stories_reaction_test['event_dttm'],infer_datetime_format=True)
stories_reaction_train = pd.read_csv(f'{data_path}stories_reaction_train.csv')
stories_reaction_train['event_dttm']=pd.to_datetime(stories_reaction_train['event_dttm'],infer_datetime_format=True)
transactions = pd.read_csv(f'{data_path}transactions.csv')


sample_submit = pd.read_csv(f'{data_path}sample_submit.csv')

In [4]:
stories_reaction_train['reaction_hour']=stories_reaction_train.event_dttm.dt.hour
stories_reaction_test['reaction_hour']=stories_reaction_test.event_dttm.dt.hour
stories_reaction_train['reaction_dayofweek']=stories_reaction_train.event_dttm.dt.dayofweek
stories_reaction_test['reaction_dayofweek']=stories_reaction_test.event_dttm.dt.dayofweek

stories_description['jsons'] = stories_description.story_json.map(lambda x:json.loads(x.replace("\\\\", "\\")))
stories_description['jsons_name'] =stories_description.jsons.map(lambda x:x['properties'].get('name'))
stories_description['texts'] =stories_description.jsons.map(lambda x:re.findall(""""text": \"([^\"]*)\",""", json.dumps(x, ensure_ascii=False)))
stories_description['jsons_name_type'] = stories_description.jsons_name.map(lambda x:jsons_name_type(x))

In [5]:
train_and_val = stories_reaction_train.merge(customer_train, on='customer_id', how='left')
train_and_val = train_and_val.merge(stories_description, on='story_id', how='left')

test = stories_reaction_test.merge(customer_test, on='customer_id', how='left')
test = test.merge(stories_description, on='story_id', how='left')

train_and_val=train_and_val.fillna(0)
# train_and_val.fillnull(0)


In [6]:
train_dates = stories_reaction_train['event_dttm'].astype(np.int64)
test_dates = stories_reaction_test['event_dttm'].astype(np.int64)
period = train_dates.max() - (test_dates.max() - test_dates.min())
val = train_and_val[train_and_val.event_dttm.astype(np.int64) > period]
train = train_and_val[train_and_val.event_dttm.astype(np.int64) <= period]

In [7]:
len(train), len(val) 

(320248, 152893)

In [8]:

train['gender_cd']=train.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
val['gender_cd']=val.gender_cd.map(lambda x: 1 if x =="M" else(0 if x =="F" else -1))
train['age']=train.age/100
val['age']=val.age/100

/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [10]:
def to_OHE(field):
    count = len(set(train_and_val[field].values))
    d = dict(zip(list(set(train_and_val[field].values)), range(1, count+1)))
    train[field]=train[field].map(lambda x:  d[x])
    val[field]=val[field].map(lambda x:  d[x])
    return count
customers_count = to_OHE('customer_id')
marital_status_count = to_OHE('marital_status_cd')
stories_count = to_OHE('story_id')
job_position_count = to_OHE('job_position_cd')
job_title_count = to_OHE('job_title')
jsons_name_type_count = to_OHE('jsons_name_type')




/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
values = set(train_and_val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values.reshape(-1))
producs_count = len(values)
d = dict(zip(list(values), range(1, len(values)+1)))
for f in ['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']:
    train[f]=train[f].map(lambda x:  d[x])
    val[f]=val[f].map(lambda x:  d[x])

/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ivan/projects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# customers_count = len(set(train_and_val.customer_id.values))
# marital_status_count = len(set(train_and_val.marital_status_cd.values))
# stories_count = len(set(train_and_val.story_id.values))
# job_position_count = len(set(train_and_val.job_position_cd.values))
# job_position_count = len(set(train_and_val.job_position_cd.values))


# customers_dict = dict(zip(sorted(list(set(train_and_val.customer_id.values))), range(1, customers_count+1)))
# stories_dict = dict(zip(sorted(list(set(train_and_val.story_id.values))), range(1, stories_count+1)))
# job_position_dict = dict(zip(sorted(list(set(train_and_val.job_position_cd.values))), range(1, job_position_count+1)))
# marital_status_dict = dict(zip(sorted(list(set(train_and_val.marital_status_cd.values))), range(1, marital_status_count+1)))

# train['customer_id']=train.customer_id.map(lambda x:  customers_dict[x])
# train['story_id']=train.story_id.map(lambda x:  stories_dict[x])
# train['job_position_cd']=train.job_position_cd.map(lambda x:  job_position_dict[x])
# train['marital_status_cd']=train.marital_status_cd.map(lambda x:  marital_status_dict[x])

# val['customer_id']=val.customer_id.map(lambda x:  customers_dict[x])
# val['story_id']=val.story_id.map(lambda x:  stories_dict[x])
# val['job_position_cd']=val.job_position_cd.map(lambda x:  job_position_dict[x])
# val['marital_status_cd']=val.marital_status_cd.map(lambda x:  marital_status_dict[x])


In [13]:
train.first_session_dttm[50000:]

75449     2014-11-09 17:35:06
75450     2014-11-09 17:35:06
75451     2014-11-09 17:35:06
75452     2014-11-05 12:28:59
75453     2014-11-05 12:28:59
                 ...         
473128                      0
473129                      0
473130                      0
473131                      0
473132                      0
Name: first_session_dttm, Length: 270248, dtype: object

In [14]:
import keras.backend as K
def metric_loss(batch_size):
    def the_loss(y_true, y_pred):

        return -(len(val.customer_id.values)/batch_size)*K.sum(y_true*y_pred)/40441.4
    return(the_loss)
# /K.mean(K.abs(y_true))
# /K.sum(K.abs(y_true))

#     return -K.sum(y_true*y_pred)/K.sum(K.abs(y_true))
#     return K.sum(y_true*[10,0.5,0.1,0.1]*y_pred)/K.sum(y_true*[-10,0.5,0.1,-0.1])

In [15]:
def reaction_to_class(reaction):
    if reaction=="dislike":
        return -10
    if reaction=="like":
        return 0.5
    if reaction=="view":
        return 0.1
    if reaction=="skip":
        return -0.1
trainY = np.array(list(train.event.map(lambda x: reaction_to_class(x)).values))
valY = np.array(list(val.event.map(lambda x: reaction_to_class(x)).values))

In [125]:
model_input_customer = Input((1,), name='customer_id')
x1 = Embedding(customers_count, 4, input_length=1)(model_input_customer)
x1 = Flatten()(x1)

model_input_story = Input((1,), name='story_id')
x2 = Embedding(stories_count, 2, input_length=1)(model_input_story)
x2 = Flatten()(x2)

# model_input_job = Input((1,), name='job_id')
# x3 = Embedding(job_position_count, 1, input_length=1)(model_input_job)
# x3 = Flatten()(x3)


model_input_job_title = Input((1,), name='job_title_id')
x4 = Embedding(job_title_count, 2, input_length=1)(model_input_job_title)
x4 = Flatten()(x4)

model_input_products = Input((7,), name='producs')
x5 = Embedding(producs_count, 1, input_length=7)(model_input_products)
x5 = Flatten()(x5)

model_input_story_name = Input((1,), name='story_name_id')
x6 = Embedding(jsons_name_type_count, 2, input_length=1)(model_input_story_name)
x6 = Flatten()(x6)

model_input_3 = Input((2,), name='customer_data')
# model_input_4 = Input((2,), name='customer_data2')

x = Concatenate()([x1, x2, x4,x5, x6, model_input_3])

x = Dense(128, 
          activation='relu', 
#           kernel_regularizer=regularizers.l2(0.0000001)
         )(x)
# x = LeakyReLU(alpha=0.1)(x)
# x = Dense(64, activation='tanh')(x)

# x = Dropout(0.5)(x)


# x = Concatenate()([x, model_input_3])

# x = Dense(4, activation='tanh')(x)


x = Dense(1, activation='tanh')(x)
model = Model([model_input_customer, 
               model_input_story, 
               model_input_story_name,
               model_input_job_title,
#                model_input_job, 
               model_input_3,
#                model_input_4, 
               model_input_products
              ], x)
# model = Model(model_input, x)





print(model.summary())

Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
customer_id (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
story_id (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
job_title_id (InputLayer)       (None, 1)            0                                            
__________________________________________________________________________________________________
producs (InputLayer)            (None, 7)            0                                            
___________________________________________________________________________________________

In [126]:
len(val.customer_id.values)

152893

In [127]:
# batch_size=320248
batch_size=32
adam = optimizers.Adam(lr=0.0001)
model.compile(optimizer=adam, loss=metric_loss(batch_size))


In [ ]:


model.fit(
    shuffle=True,
#     x = [train.customer_id.values, train.story_id.values, train[['age', 'gender_cd']].values],
        x = [train.customer_id.values,
             train.story_id.values,
             train.jsons_name_type.values, 
             train.job_title.values,
#              train.job_position_cd.values,
             train[['age', 'gender_cd']].values,
#              train[['reaction_hour', 'reaction_dayofweek']].values/30,
             train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
            ], 
# steps_per_epoch=1000,
          y = trainY,
          batch_size=batch_size,
          epochs=30,
#     validation_data = ([val.customer_id.values, val.story_id.values, val[['age', 'gender_cd']].values], valY),
          validation_data=([val.customer_id.values, 
                            val.story_id.values, 
                            val.jsons_name_type.values, 
                            val.job_title.values,
#                             val.job_position_cd.values,
                            val[['age', 'gender_cd']].values,
#                             val[['reaction_hour', 'reaction_dayofweek']].values/30,
                            val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
                           ], valY),
         )

# 320248/320248 [==============================] - 13s 41us/step - loss: -0.6153 - val_loss: -0.3492
# 320248/320248 [==============================] - 13s 42us/step - loss: -0.7373 - val_loss: -0.3607
# 320248/320248 [==============================] - 14s 42us/step - loss: -0.7263 - val_loss: -0.3656 size 4 tanh
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.7046 - val_loss: -0.3612 size 128 relu
# 320248/320248 [==============================] - 14s 44us/step - loss: -0.7110 - val_loss: -0.3455 size 128 lrelu
# 320248/320248 [==============================] - 14s 43us/step - loss: -0.6390 - val_loss: -0.3683 size 128 relu increase embs
# 320248/320248 [==============================] - 16s 48us/step - loss: -0.6319 - val_loss: -0.3705 job_position_cd




Train on 320248 samples, validate on 152893 samples
Epoch 1/30
320248/320248 [==============================] - 33s 102us/step - loss: -0.2163 - val_loss: -0.2293
Epoch 2/30
320248/320248 [==============================] - 32s 100us/step - loss: -0.3366 - val_loss: -0.3234
Epoch 3/30
225792/320248 [====================>.........] - ETA: 7s - loss: -0.5276

In [ ]:
train_predict = model.predict([train.customer_id.values,
                               train.story_id.values,
                               train.jsons_name_type.values, 

                               train.job_title.values, 
                               train[['age', 'gender_cd']].values,
                               train[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
                              ])
val_predict = model.predict([val.customer_id.values,
                             val.story_id.values,
                             val.jsons_name_type.values,
                             val.job_title.values,
#                              val.job_position_cd.values,
                             val[['age', 'gender_cd']].values,
#                              val[['reaction_hour', 'reaction_dayofweek']].values/30,
                             val[['product_0', 'product_1', 'product_2', 'product_3', 'product_4', 'product_5', 'product_6']].values
                            ])
# train_predict = model.predict(train.customer_id.values)
# val_predict = model.predict(val.customer_id.values)

In [37]:
np.sum(np.abs(trainY))

94640.0

In [38]:
-np.sum(trainY*train_predict[:,0])/np.sum(np.abs(trainY))

-0.6840569814593728

In [39]:
# -np.sum(trainY*train_predict[:,0] )/np.sum(np.abs(trainY))


In [40]:
-np.sum(valY*val_predict[:,0] )/np.sum(np.abs(valY))
# -0.22921870322866542
# -0.29499408463224436
# -0.3365937438413243
# -0.34325145608255186
# -0.35394938525789055

-0.3590916611807316

In [26]:
val['predict'] = val_predict[:,0]

/home/ivan/PycharmProjects/tks_stories/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
val.loc[~val.customer_id.isin(set(train.customer_id.values)),'predict'] =-1

/home/ivan/PycharmProjects/tks_stories/venv/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [28]:
-np.sum(valY*val['predict'])/np.sum(np.abs(valY))
# -0.26261507362945524
# -0.29364859095899104
# -0.3315410025698421

-0.31330923716729303